In [2]:
import os

In [3]:
%pwd

'/Users/sunilverma/Text-Summarizer-Project/research'

In [4]:
os.chdir('/Users/sunilverma/Text-Summarizer-Project')

In [5]:
%pwd

'/Users/sunilverma/Text-Summarizer-Project'

In [7]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class DataValidationConfig: 
    root_dir: Path 
    status_file: Path
    All_REQUIRED_FILES: list

In [8]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        print(f"DEBUG - ConfigurationManager received config_filepath: {config_filepath}")
        print(f"DEBUG - ConfigurationManager received params_filepath: {params_filepath}")
        
        # Convert relative paths to absolute
        if not config_filepath.is_absolute():
            config_filepath = Path(os.getcwd()) / config_filepath
        if not params_filepath.is_absolute():
            params_filepath = Path(os.getcwd()) / params_filepath
        
        print(f"DEBUG - After path conversion, config_filepath: {config_filepath}")
        print(f"DEBUG - After path conversion, params_filepath: {params_filepath}")
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        create_directories([config.root_dir]) 
        data_validation_config = DataValidationConfig(
            root_dir= config.root_dir,
            status_file=config.STATUS_FILE,
            All_REQUIRED_FILES=config.REQUIRED_FILES,
        )
        return data_validation_config

In [10]:
import os
from text_summarizer.logging import logger

In [12]:
class DataValidation: 
    def __init__(self, config: DataValidationConfig):
        self.config = config
    
    def validate_all_files_exist(self) -> bool:
        try: 
            validation_status = True

            all_files = os.listdir(os.path.join('artifacts',"data_ingestion", "samsum_dataset"))

            for file in all_files: 
                if file not in self.config.All_REQUIRED_FILES: 
                    validation_status = False
                    with open(self.config.status_file, 'w') as f: 
                        f.write(f"valiation status: {validation_status}")
                else: 
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"valiation status: {validation_status}")
            return validation_status
        except Exception as e:
            raise e

In [13]:
try: 
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config() 
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

DEBUG - ConfigurationManager received config_filepath: config/config.yaml
DEBUG - ConfigurationManager received params_filepath: params.yaml
DEBUG - After path conversion, config_filepath: /Users/sunilverma/Text-Summarizer-Project/config/config.yaml
DEBUG - After path conversion, params_filepath: /Users/sunilverma/Text-Summarizer-Project/params.yaml
2025-12-17 20:49:46,281: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/config/config.yaml loaded successfully
2025-12-17 20:49:46,290: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/params.yaml loaded successfully
2025-12-17 20:49:46,292: INFO: common: created directory at: artifacts
2025-12-17 20:49:46,293: INFO: common: created directory at: artifacts/data_validation
